# 1 Data Preparation and Loading

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/car_vs_ambulance_classification_ensemble_cnn'

In [9]:
# automatically reload all modules before executing
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.config.list_physical_devices('GPU'), tf.__version__

2024-09-25 17:17:45.743644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 17:17:45.754517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 17:17:45.757975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 17:17:45.767028: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1727284667.290848    7851 cuda_executor.c

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.17.0')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from utils import utils
import opendatasets as od
import random
import pathlib 
import os
import glob
import cv2

In [10]:
params_config_path= pathlib.Path('params.yaml')
params_config = utils.read_yaml(yaml_path=params_config_path)
params_config

2024-09-25 17:21:37,642 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully


ConfigBox({'base': {'random_seed': 42}, 'data_split': {'test_size': 0.2}, 'train': {'image_size': 224, 'batch_size': 32, 'fine_tune_args': {'epochs': 10}, 'letnet': {'base_lr': 0.01}, 'mini_vgg_net': {'base_lr': 0.01}, 'shallownet': {'base_lr': 0.01}}, 'evaluate': {'reports_dir': 'reports', 'metrics_file': 'metrics.json', 'confusion_matrix_image': 'confusion_matrix.png', 'confusion_matrix_data': 'confusion_matrix_data.csv'}, 'paths': {'dataset_url': 'https://www.kaggle.com/datasets/amansingh0000000/image-classification-and-yolo-dataset/data', 'dataset_dir': 'dataset/raw', 'dataset_path': 'dataset/image-classification-and-yolo-dataset/Ambulance_Car', 'preprocess_dataset': 'dataset/preprocess', 'train_dir': 'dataset/raw/image-classification-and-yolo-dataset/Ambulance_Car/Train', 'labels_dir': 'dataset/raw/image-classification-and-yolo-dataset/Ambulance_Car/Label', 'preprocessed_train': '/workspaces/car_vs_ambulance_classification_ensemble_cnn/dataset/preprocess/train', 'preprocessed_test

In [11]:
tf.random.set_seed(params_config.base.random_seed)

In [12]:
train_dir = params_config.paths.preprocessed_train
val_dir = params_config.paths.preprocessed_val

In [13]:
train_dir

'/workspaces/car_vs_ambulance_classification_ensemble_cnn/dataset/preprocess/train'

In [16]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  subset="training",
  validation_split=0.2,
  seed=params_config.base.random_seed,
  image_size=(params_config.train.image_size, params_config.train.image_size),
  batch_size=params_config.train.batch_size)

Found 2227 files belonging to 2 classes.
Using 1782 files for training.


I0000 00:00:1727285188.619630    7851 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727285188.620951    7851 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727285188.621928    7851 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727285188.734439    7851 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [17]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  subset="validation",
  validation_split=0.2,
  seed=params_config.base.random_seed,
  image_size=(params_config.train.image_size, params_config.train.image_size),
  batch_size=params_config.train.batch_size)

Found 557 files belonging to 2 classes.
Using 111 files for validation.


# 2 Model Training

In [4]:
from src.models import lenet, mini_vgg_net, shallownet
from tensorflow.keras.optimizers import SGD

2024-09-24 17:23:32.540116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 17:23:32.616068: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 17:23:32.637327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 17:23:32.784490: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
lenet_model = lenet.LeNet()
mini_vgg_net_model =mini_vgg_net.MiniVGGNet()
shallownet_model = shallownet.ShallowNet()

In [ ]:
lenet_model_opt = SGD(learning_rate=1e-2, weight_decay=0.01 / 40, momentum=0.9, nesterov=True)
lenet_model = lenet_model.build(width=params_config.train.image_size, 
                                          height=params_config.train.image_size, 
                                          depth=3)
lenet_model.summary()

### LeNet Train

In [ ]:
lenet_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "precision", "recall"])

# train the network
Lenet_model_history = lenet_model.fit(train_dataset_prefetch,
              validation_data=val_dataset_prefech,
              batch_size=params_config.train.batch_size,
              epochs=20,
              verbose=1)